# Working with spatially enabled dataframes

In this example, we're going to go through loading data as a spatially-enabled dataframe using the [Pandas](https://pandas.pydata.org/docs/) library. 

# Import libraries
We'll import arcpy, arcgis.gis from the ArcGIS API for Python, and import Pandas with the alias pd as per convention.

In [1]:
import arcpy
from arcgis.gis import GIS
import pandas as pd

# Joining a shapefile with a table from a csv file
This example is an equivalent to the '[Join Field](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/join-field.htm)' geoprocessing tool. We'll read in a shapefile as a spatially-enabled dataframe, and read in a csv file as a regular Pandas dataframe, then join the two using the Pandas `.merge` function. 

First, we'll read in the shapefile and convert it to a spatially-enabled dataframe. Make sure you've downloaded and extracted the AHAH data for Bristol from https://data.cdrc.ac.uk/geodata-packs to the python_data folder in your project folder.

In [2]:
# Replace the path with a relative path
file = r".\python_data\Access_to_Healthy_Assets_and_Hazards_AHAH\Local_Authority_Districts\E06000023\shapefiles\E06000023.shp"
LSOA = pd.DataFrame.spatial.from_featureclass(file)
LSOA.head()

,FID,lsoa11cd,SHAPE
0,0,E01014485,"{""rings"": [[[360256.271, 174422.695], [360255,..."
1,1,E01014486,"{""rings"": [[[359676.043, 174114.74], [359661.3..."
2,2,E01014487,"{""rings"": [[[359036.633, 174653.14], [359035.2..."
3,3,E01014488,"{""rings"": [[[359632.58, 174806.408], [359634.1..."
4,4,E01014489,"{""rings"": [[[359120.567, 174097.45], [359120.3..."


Let's take a look at the number of rows and columns in the dataframe by using the pandas `.shape` function.

In [3]:
LSOA.shape

(263, 3)

Next, we'll read in a csv file as a 'regular' dataframe using `pd.read_csv()` and take a look at the first 5 rows with the Pandas `.head()` function.

In [3]:
df = pd.read_csv(r"python_data\Access_to_Healthy_Assets_and_Hazards_AHAH\Local_Authority_Districts\E06000023\tables\E06000023.csv")
df.head()

,lsoa11cd,lsoa11nm,r_rank,h_rank,g_rank,e_rank,r_exp,h_exp,g_exp,e_exp,ahah,r_ahah,d_ahah,r_dec,h_dec,g_dec,e_dec,gpp_dist,ed_dist,dent_dist,pharm_dist,gamb_dist,ffood_dist,pubs_dist,leis_dist,blue_dist,off_dist,tobac_dist,green_pas,green_act,ur,no2_mean,pm10_mean,so2_mean,d_gpp_dist,d_ed_dist,d_pharm_dist,d_dent_dist,d_gamb_dist,d_ffood_dist,d_pubs_dist,d_leis_dist,d_blue_dist,d_off_dist,d_tobac_dist,d_green_pas,d_green_act,d_no2_mean,d_pm10_mean,d_so2_mean
0,E01014485,Bristol 023A,35286,7682,1724,28394,41.394453,4.612424,0.957596,25.613595,18.144517,15309,4,9,2,1,7,0.997593,2.736111,0.817963,1.057593,0.817963,0.277407,0.231852,0.396481,2.051667,2.027593,2.295185,1.299151,0.180,1,13.573317,13.376697,1.331312,5,2,7,4,7,10,10,1,5,6,6,1,1,8,6,8
1,E01014486,Bristol 023B,34993,5387,4195,28476,40.449998,3.135055,2.403604,25.749859,17.934629,14817,4,9,2,2,7,0.858382,2.110882,0.568676,0.853235,0.568676,0.352500,0.396029,0.545147,2.272500,1.798235,1.871618,0.289311,0.120,1,13.982219,13.297512,1.307590,4,1,5,2,9,9,9,1,6,7,7,5,1,8,6,7
2,E01014487,Bristol 020A,36285,6117,19801,27722,44.941656,3.594770,14.531847,24.526307,21.898645,23835,6,9,2,5,7,0.814828,1.715172,0.773448,0.713448,0.627931,0.540345,0.560345,0.829655,2.879655,0.800690,0.860345,0.194665,0.290,1,13.530668,13.257475,1.307112,4,1,4,4,8,8,8,2,7,9,10,6,2,8,6,7
3,E01014488,Bristol 023C,35462,5694,18682,28449,41.980996,3.327268,13.414295,25.704902,21.106865,22175,6,9,2,5,7,0.854318,1.611818,0.606364,0.884773,0.564545,0.494545,0.351136,0.705455,2.723864,1.461591,1.530682,0.203573,0.285,1,13.666415,13.360288,1.326760,4,1,6,2,9,9,10,1,7,7,8,6,2,8,6,8
4,E01014489,Bristol 023D,39888,654,9202,28602,66.098068,0.358587,5.645883,25.960826,24.515841,28788,7,10,1,3,7,0.385532,1.004894,0.632553,0.431489,0.571064,0.205532,0.149362,0.337021,1.901489,0.987021,1.026383,0.198259,0.200,1,14.008808,13.316356,1.312781,1,1,2,3,9,10,10,1,5,9,9,6,1,8,6,7


Let's check the shape of this dataframe too.

In [5]:
df.shape

(263, 50)

We've got the same number of rows in this data table (one for each LSOA), but there are 50 columns. While we could merge all of these columns with our shapefile, it's tidier to just merge in the columns we want. To do this, we'll create a new subset dataframe called 'join_data' containing just the field we're going to join on (the LSOA code, `'lsoa11cd'`) and our variables of interest - in this example, `'no2_mean'`, `'pm10_mean'`, and `'so2_mean'`.

In [6]:
join_data = df[['lsoa11cd', 'no2_mean', 'pm10_mean', 'so2_mean']]
join_data.head()

,lsoa11cd,no2_mean,pm10_mean,so2_mean
0,E01014485,13.573317,13.376697,1.331312
1,E01014486,13.982219,13.297512,1.307590
2,E01014487,13.530668,13.257475,1.307112
3,E01014488,13.666415,13.360288,1.326760
4,E01014489,14.008808,13.316356,1.312781


We can now join the two data sources based on the `lsoa11cd` field using Pandas `.merge()` function.

In [7]:
join_df = LSOA.merge(join_data, on="lsoa11cd", how='left')
join_df.head()

,FID,lsoa11cd,SHAPE,no2_mean,pm10_mean,so2_mean
0,0,E01014485,"{'rings': [[[360256.271, 174422.695], [360255,...",13.573317,13.376697,1.331312
1,1,E01014486,"{'rings': [[[359676.043, 174114.74], [359661.3...",13.982219,13.297512,1.307590
2,2,E01014487,"{'rings': [[[359036.633, 174653.14], [359035.2...",13.530668,13.257475,1.307112
3,3,E01014488,"{'rings': [[[359632.58, 174806.408], [359634.1...",13.666415,13.360288,1.326760
4,4,E01014489,"{'rings': [[[359120.567, 174097.45], [359120.3...",14.008808,13.316356,1.312781


We can now save our new spatially enabled dataframe to our project geodatabase as a new feature class.

In [8]:
# TODO: Update the name of the geodatabase as required.
join_df.copy().spatial.to_featureclass(location=r".\Dataviz_ArcGIS_API_for_Python.gdb\AHAH_AQ_Bristol")

'C:\\EsriTraining\\VisualPythonAPI\\VisualPythonAPI\\Dataviz_ArcGIS_API_for_Python.gdb\\AHAH_AQ_Bristol'

In [10]:
join_df.copy().spatial.to_featureclass(location=r".\Dataviz_ArcGIS_API_for_Python.gdb\AHAH_AQ_Bristol")

In [15]:
# removes locks on database after writing files. TODO: Look for alternate methods, and root cause of lock.
arcpy.management.Compact(r"C:\EsriTraining\VisualPythonAPI\VisualPythonAPI\Dataviz_ArcGIS_API_for_Python.gdb")

<Result 'C:\\EsriTraining\\VisualPythonAPI\\VisualPythonAPI\\Dataviz_ArcGIS_API_for_Python.gdb'>

In [12]:
# Update the name of the geodatabase as required.
join_df.spatial.to_featureclass(location=r".\Dataviz_ArcGIS_API_for_Python.gdb\AHAH_NonCopyTest")

'C:\\EsriTraining\\VisualPythonAPI\\VisualPythonAPI\\Dataviz_ArcGIS_API_for_Python.gdb\\AHAH_NonCopyTest'

 We can also export the spatially enabled dataframe as a shapefile if we save it outside a geodatabase and specify the extension '.shp'.

In [16]:
# Export to shapefile:
join_df.spatial.to_featureclass(location=r".\python_data\AHAH_Bristol.shp")

'C:\\EsriTraining\\VisualPythonAPI\\VisualPythonAPI\\python_data\\AHAH_Bristol.shp'

## Reading in point data from a csv file

We can read in a table of points with an X and Y column as a spatially enabled dataframe by loading it as a 'regular' pandas dataframe, then passing it to the `pd.DataFrame.spatial.from_xy()` function.

In [2]:
df = pd.read_csv(r'.\dft-road-casualty-statistics-accident-2020.csv') # for example
sedf = pd.DataFrame.spatial.from_xy(df,'longitude','latitude') # specify cols for X and Y
sedf.head()

sys:1: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.


,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,SHAPE
0,2020010219808,2020,10219808,521389.0,175144.0,-0.254001,51.462262,1,3,1,1,04/02/2020,3,09:00,10,E09000032,E09000032,6,0,6,20,0,-1,0,-1,9,9,1,9,9,0,0,1,3,2,E01004576,"{""spatialReference"": {""wkid"": 4326}, ""x"": -0.2..."
1,2020010220496,2020,10220496,529337.0,176237.0,-0.139253,51.470327,1,3,1,2,27/04/2020,2,13:55,9,E09000022,E09000022,3,3036,6,20,9,2,6,0,0,4,1,1,1,0,0,1,1,2,E01003034,"{""spatialReference"": {""wkid"": 4326}, ""x"": -0.1..."
2,2020010228005,2020,10228005,526432.0,182761.0,-0.178719,51.529614,1,3,1,1,01/01/2020,4,01:25,1,E09000033,E09000033,5,0,6,30,3,1,6,0,0,0,4,1,2,0,0,1,1,2,E01004726,"{""spatialReference"": {""wkid"": 4326}, ""x"": -0.1..."
3,2020010228006,2020,10228006,538676.0,184371.0,-0.001683,51.541210,1,2,1,1,01/01/2020,4,01:50,17,E09000025,E09000025,3,11,6,30,0,-1,0,-1,0,4,4,1,1,0,0,1,1,2,E01003617,"{""spatialReference"": {""wkid"": 4326}, ""x"": -0.0..."
4,2020010228011,2020,10228011,529324.0,181286.0,-0.137592,51.515704,1,3,1,2,01/01/2020,4,02:25,1,E09000033,E09000033,3,40,6,30,3,4,5,0,0,0,4,1,1,0,0,1,1,2,E01004763,"{""spatialReference"": {""wkid"": 4326}, ""x"": -0.1..."


We can subset a Pandas dataframe using the Pandas `.query()` function. An example could be:

`subset_dataframe = dataframe.query('field_name == "something_in_field_name"')`

or 

`subset_dataframe = dataframe.query('field_name <= 70')`

Remember to use the equality operator (`==`), not the assignment operator (`=`).

In [43]:
sedf2 = sedf.query('lsoa_of_accident_location == "E01033355"')

In [47]:
sedf2.info

<bound method DataFrame.info of       accident_index  ...                                              SHAPE
80338  2020522000484  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
80387  2020522000964  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
80428  2020522001482  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
80472  2020522001780  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
80599  2020522002300  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
80610  2020522002347  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
80686  2020522002660  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
80699  2020522002691  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
80737  2020522002919  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
80873  2020522003657  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
81058  2020522004424  ...  {"spatialReference": {"wkid": 4326}, "x": -2.5...
81562  2020522101056  ...  {"spatialReferenc

In [44]:
sedf2.spatial.plot() # Try using .loc[row_indexer,col_indexer] = value instead

MapView(layout=Layout(height='400px', width='100%'))